#1 Carga y limpieza de datos


In [28]:
#Archivo a trabajar
import csv

file = 'EEG_eyes_classification.csv'
file_filtered = 'filtered_file.csv'
"""
Informacion del archivo:
-Class: b'1' = 0; b'2' = 1 y valores atipicos
Manejar valores nulos, identificar cada una de las columnas y corregir lo que se encuentre.
HINT: calcular el porcentaje de valores por columna es de gran ayuda, sin tener miedo a eliminar lo que no sirve.
"""

def calcular_porcentaje_vacios(fila):
    # Calcula el número de valores vacíos en la fila
    num_vacios = sum(1 for valor in fila if not valor.strip())
    # Calcula el porcentaje de valores vacíos
    porcentaje_vacios = (num_vacios / len(fila))
    return porcentaje_vacios

#Se filtran las columnas con mas de un dato faltante
cont1 = 0
cont0 = 0
with open(file, 'r') as csvfile, open(file_filtered, 'w', newline='') as newcsv:
    reader = csv.reader(csvfile, delimiter=',')
    writer = csv.writer(newcsv)
    for row in reader:
        porcentaje_vacio = calcular_porcentaje_vacios(row)

        if porcentaje_vacio < 2/16:
            if row[-1] == "b'1'":
                cont0 += 1
                row[-1] = 0
            elif row[-1] == "b'2'":
                cont1 += 1
                row[-1] = 1
            elif row[-1] == "Open":
                cont0 += 1
                row[-1] = 0
            if row[-1] == "Close" or row[-1] == "Closed":
                cont1 += 1
                row[-1] = 1
            writer.writerow(row)

#2 Balanceo de datos


In [29]:
print(cont0, cont1)
file_2 = 'filtered_2.csv'
zeros = 0
ones = 0
with open(file_filtered, 'r') as csvfile, open(file_2, 'w', newline='') as newcsv:
    reader = csv.reader(csvfile, delimiter=',')
    writer = csv.writer(newcsv)
    for row in reader:
        if row[-1] == 'Class':
            writer.writerow(row)
        if cont0 > cont1:
            if row[-1] == '1':
                writer.writerow(row)
                cont1+=1
                ones +=1
        elif row[-1] == '0' and zeros <= ones:
            zeros+=1
            writer.writerow(row)

zeros = 0
ones = 0
with open(file_2, 'r') as file2:
    reader = csv.reader(file2, delimiter=',')
    for row in reader:
        if row[-1] == '0':
            zeros+=1
        elif row[-1] == '1':
            ones+=1
print(zeros,ones)

7993 6493
1501 1500
